In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
my_data = pd.read_csv('Data.csv')

In [3]:
my_data

,Sex,Age_category,Highest_education_level
0,2,2,2
1,1,2,3
2,2,1,2
3,2,2,3
4,1,2,2
...,...,...,...
195,2,1,1
196,1,2,2
197,2,1,3
198,1,1,0


In [4]:
my_data_corr = my_data.corr()
my_data_corr

,Sex,Age_category,Highest_education_level
Sex,1.000000,-0.015917,-0.069140
Age_category,-0.015917,1.000000,0.239223
Highest_education_level,-0.069140,0.239223,1.000000


In [5]:
from sklearn.linear_model import LinearRegression

In [6]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression

model_accuracy = []

for i in range(3):
    featured_data = [x for x in my_data.columns if x != my_data.columns[i]]
    target_data = my_data.columns[i]
    X = my_data[featured_data].values
    y = my_data[target_data].values  # Correct assignment of the target variable
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    model_accuracy.append(model.score(X_test, y_test))

model_accuracy




[0.0009731393029177582, 0.06105471905240423, 0.030194634679886034]

In [7]:
from sklearn.neighbors import KernelDensity 

kde = KernelDensity(bandwidth=0.1, kernel='gaussian')  # Adjust bandwidth and kernel as needed

# Fit the KDE model to your data
kde.fit(X)

# Sample from the KDE model
# You can generate synthetic data by sampling from the KDE model
synthetic_data = kde.sample(n_samples=50000, random_state=42)

synthetic_data

array([[0.99809366, 1.12072883],
       [2.06252924, 0.9195311 ],
       [1.17871134, 1.84641735],
       ...,
       [0.91940236, 0.93610753],
       [1.07405388, 1.96773031],
       [1.96529718, 2.16863405]])

In [9]:
pip install copulas


  Obtaining dependency information for copulas from https://files.pythonhosted.org/packages/0d/c2/366534be86ef6294fd8b75e0762037eb169962fd437afbbbd3b0248c7baa/copulas-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for plotly>=5.10.0 from https://files.pythonhosted.org/packages/00/4e/6258fc3b26f1f7abd1b2e75b1e9e4f12f13584136e2e1549f995ff4c6b7b/plotly-5.20.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/51.9 kB ? eta -:--:--
   ------- -------------------------------- 10.2/51.9 kB ? eta -:--:--
   ---------------------------------------- 51.9/51.9 kB 887.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.7 MB 2.6 MB/s eta 0:00:06
   - -------------------------------------- 0.4/15.7 MB 5.0 MB/s eta 0:00:04
   - -------------------------------------- 0.7/15.7 MB 6.4 MB/s eta 0:00:03
   -- ------------------------------------- 1.0/15.7 MB 5.8 MB/s eta 0:0

In [12]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from copulas.multivariate import GaussianMultivariate

# Step 1: Estimate marginal distributions
marginal_distributions = {}
for column in my_data.columns:
    kde = gaussian_kde(my_data[column])
    marginal_distributions[column] = kde

# Step 2: Fit a copula model
copula = GaussianMultivariate()
copula.fit(my_data)

# Step 3: Generate synthetic copula data
synthetic_copula_data = copula.sample(num_samples)

# Step 4: Transform synthetic copula data
synthetic_data = {}
for column in my_data.columns:
    # Approximate inverse CDF using integrate_box_1d
    def inverse_cdf(u):
        return marginal_distributions[column].integrate_box_1d(0, u)

    # Apply inverse CDF to synthetic copula data
    synthetic_data[column] = np.array([inverse_cdf(u) for u in synthetic_copula_data[column]])

# Convert synthetic data to DataFrame
synthetic_df = pd.DataFrame(synthetic_data)

C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)


In [13]:
synthetic_df

,Sex,Age_category,Highest_education_level
0,0.996927,0.494366,0.473001
1,0.878542,0.541014,0.355634
2,0.929985,0.965192,0.627371
3,0.391270,0.264366,0.527208
4,0.591026,0.464075,0.399376
...,...,...,...
49995,0.563129,0.487849,0.368299
49996,0.439742,0.860026,0.691818
49997,0.635635,0.866235,0.349410
49998,0.128611,0.148816,-0.072047


In [14]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
from copulas.multivariate import GaussianMultivariate

# Define mappings for each column
sex_mapping = {1: 'male', 2: 'female'}
age_category_mapping = {1: 'young', 2: 'middle-aged', 3: 'old'}
highest_education_level_mapping = {0: 'none', 1: 'primary', 2: 'secondary', 3: 'tertiary'}

# Step 1: Estimate marginal distributions
marginal_distributions = {}
for column in my_data.columns:
    kde = gaussian_kde(my_data[column])
    marginal_distributions[column] = kde

# Step 2: Fit a copula model
copula = GaussianMultivariate()
copula.fit(my_data)

# Step 3: Generate synthetic copula data
synthetic_copula_data = copula.sample(num_samples)

# Step 4: Transform synthetic copula data
synthetic_data = {}
for column in my_data.columns:
    # Approximate inverse CDF using integrate_box_1d
    def inverse_cdf(u):
        return marginal_distributions[column].integrate_box_1d(0, u)

    # Apply inverse CDF to synthetic copula data
    transformed_values = np.array([inverse_cdf(u) for u in synthetic_copula_data[column]])

    # Calculate cumulative probabilities for integer categories
    unique_values = np.sort(my_data[column].unique())
    cumulative_probs = np.array([marginal_distributions[column].integrate_box_1d(-np.inf, value) for value in unique_values])
    cumulative_probs /= cumulative_probs.sum()

    # Map transformed values to integer categories based on cumulative probabilities
    synthetic_data[column] = np.array([unique_values[np.argmax(cumulative_probs >= value)] for value in transformed_values])

C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:719: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
C:\Users\divij\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:5999: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)


In [15]:
synthetic_data

{'Sex': array([1, 1, 2, ..., 1, 2, 2], dtype=int64),
 'Age_category': array([1, 2, 2, ..., 1, 1, 2], dtype=int64),
 'Highest_education_level': array([0, 0, 3, ..., 0, 2, 1], dtype=int64)}

In [16]:




# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(synthetic_data)

# Print the DataFrame
print(df)


       Sex  Age_category  Highest_education_level
0        1             1                        0
1        1             2                        0
2        2             2                        3
3        2             1                        0
4        2             1                        0
...    ...           ...                      ...
49995    1             1                        3
49996    2             1                        0
49997    1             1                        0
49998    2             1                        2
49999    2             2                        1

[50000 rows x 3 columns]


In [17]:
df['Age_category'].unique()

array([1, 2, 3], dtype=int64)

In [18]:
df.corr()

,Sex,Age_category,Highest_education_level
Sex,1.000000,0.005152,-0.003165
Age_category,0.005152,1.000000,0.020841
Highest_education_level,-0.003165,0.020841,1.000000
